In [5]:
__author__ = "Miguel Solinas ","Elvin Ugonna"
__credits__ = "Miguel Solinas ","Elvin Ugonna"
__version__ = "0.1.0"
__maintainer__ = "Miguel Solinas Jr.","Elvin Ugonna"
__email__ = "migue.solinas@gmail.com","elvindavin@gmail.com"
__status__ = "Project"

In [6]:
    #importing libraries
import pandas as pd
import numpy as np

In [7]:
#open dataset
file = pd.read_csv("data1.csv")
dataset = pd.DataFrame(file)

dataset.dtypes

FileNotFoundError: File b'data1.csv' does not exist

In [2]:
dataset.classs[dataset.classs==1]=0
dataset.classs[dataset.classs==2]=1
dataset.to_csv("dataset.csv")
dataset.describe()
dataset.describe()

NameError: name 'dataset' is not defined

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="white")

# Generate a large random dataset


# Compute the correlation matrix
corr = dataset.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})



print(" 1) Some of the featues do not deliver major information\n") 
print(" 2) The feature type is usefull, it contains just one value (4)\n")
print(" 3) Pos/2 is not correlated to any feature and it does not change along the samples it is always close to one value\n ")
print(" 4) I can not ensure if there is a correlation between the feature class and the rest ot the features \n")
print(" 5) Dataset not well balanced clas1 = ", np.count_nonzero(dataset.classs==0), "class2 =",np.count_nonzero(dataset.classs==1))

NameError: name 'dataset' is not defined

In [6]:
#  Descrpition:
#  Set of normalization functions
#  Normalize each columns of the dataset, this function accept a normalize function like argument (lambda : function()).
#  Args:
#   dataframe: pandas DataFrame of features
#   Normalize function: function to be implemented to normalize the dataset
#  Returns:
#    A version of the input `DataFrame` that has all its features normalized following the specified function.

def linear_scale(series):
  min_val = series.min()
  max_val = series.max()
  scale = (max_val - min_val) / 2.0
  return series.apply(lambda x:((x - min_val) / scale) - 1.0)

def z_score_normalize(series):
  mean = series.mean()
  std_dv = series.std()
  return series.apply(lambda x:(x - mean) / std_dv)

def get_quantile_based_boundaries(feature_values, num_buckets):
  boundaries = np.arange(1.0, num_buckets) / num_buckets
  quantiles = feature_values.quantile(boundaries)
  return [quantiles[q] for q in quantiles.keys()]

def feature_normalize(dataset):
    mu = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    return (dataset - mu)/sigma

def min_max(dataset):
    mu = np.mean(dataset,axis=0)
    max_ = np.max(dataset)
    min_ = np.min(dataset)
    return (dataset - mu)/(max_-min_)



In [7]:
# We need to split the dataset into three (validation, training and testing)
# They must to be the same everytime, we can respect that by choosing a random seed (in this case 1)

def split_dataset (dataframe,validation=False,seed=1):
    np.random.seed(seed) 
    mask1 = np.random.randn(len(dataframe))<0.7
    training = dataframe[mask1]
    no_training = dataframe[~mask1]
    if validation :
        mask2 = np.random.randn(len(no_training))>0.5
        validation = no_training[mask2]
        testing = no_training[~mask2] # ~ gives the complement of a binary chipher from 0100 to 1011 (https://data-flair.training/blogs/python-operator/)
        return training, validation, testing
    else:
        print("validation equals to testing")
        return training, no_training,no_training
    
    

In [8]:
from tensorflow.python.data import Dataset
#The tf.data API enables you to build complex input pipelines from simple, reusable pieces


class dataset_setup ():

    def __init__(self,features,targets):
        self.allDataset_features = None
        self.allDataset_targets = None
        self.training_features = None
        self.training_targets = None
        self.validation_features = None
        self.validation_targets= None
        self.testing_features = None
        self.testing_targets= None
        self.targets= targets
        self.features= features
        self.targets_aux= None
        self.features_aux= None
        
    def input_data(self,path_file,sep=',',header = 0):
        
        dataset_original = pd.read_csv(path_file, sep=sep, header=header)
#         pd.options.display.float_format = '{:.4f}'.format
        
        dataset = pd.DataFrame(dataset_original.reindex(
            np.random.permutation(dataset_original.index)))
        
        #Normalization
        dataset[features] = min_max(dataset[features]) # we need to normalize just the features not the labels
        #Splitting dataset into three subsets    
        ds_training, ds_validation, ds_testing = split_dataset(dataset,validation =True)
        
        print("Training samples ",ds_training.shape,"\nValidation samples",ds_validation.shape,"\nTesting samples",ds_testing.shape)
        #All dataset features and targets
        self.allDataset_features = dataset[self.features]
        self.allDataset_targets = dataset[self.targets]
        #Training dataset
        self.training_features = ds_training[self.features]
        self.training_targets = ds_training[self.targets]
        #Validation dataset
        self.validation_features =  ds_validation[self.features]
        self.validation_targets = ds_validation[self.targets]
        #Testing dataset
        self.testing_features = ds_testing[self.features]
        self.testing_targets = ds_testing[self.targets]

# Name: 
#  pipeline_feeder
# Description:
#  The function will receive a couple of features/targets (dataframes) that will be splitted into a specific
#  number of epoch. Each epoch will containt as samples as the batchsize specified.
# Args:
#  features
#   type  -> DataFrame
#   features -> It is expected a dataframe with the columns of the features and the respective data 
#   targets  -> It is expected a dataframe with the labels. labels can be coded in one-hote or discrete 
#           -> n° labels = n° targets  they must contain the same index position (Sample 1 feature -> sample 1 target) 
#  Batch_size:
#   type  -> Integer
#   It defines the size of the batch, if batch_size is None the dataset will not be splitted (Batch gradient vs mini-batch (S.G.D))
#  Features_aux:
#   type  -> DataFrame
#   Extra features to be splitted if it is required 
#  targets_aux:
#   type  -> DataFrame
#   Extra target to be splitted if it is required # shuffle
#  Shuffle:
#   type  -> Boolean
#   The batch can be shuffled to increase randomization
#  num_epochs:
#   type  -> Integer
#   It defines the times that a batch must be loaded in memory
# Returns:
#  A  couple of target/features iterators

    def pipeline_feeder(self,features, targets, batch_size,features_aux= None,targets_aux=None, shuffle = False, num_epochs=None):


        # warning: 2GB limit
        if features_aux is not None: 
            features_new = [features,features_aux] #we create a list containing extra features or extra targets
            features = pd.concat(features_new,1) #we concatenate the extra information with the original one alongisde index 1 
        elif targets_aux is not None:
            targets_new = [features,features_aux]
            target = pd.concat(targets_new,1)

        print(features.shape, targets.shape)
        ds = Dataset.from_tensor_slices((features, targets))  # We call dataset function from the API data that allow us to create
                                                              # tensor objects to feed the computational graph.

        # Shuffle the data, if specified
        if shuffle:
            ds = ds.shuffle(10000)

        if batch_size == 0:
            ds = ds.batch(features.shape[0]).repeat(num_epochs)
        else:
            ds = ds.batch(batch_size).repeat(num_epochs)

        # Return the next batch of data
        features, labels = ds.make_one_shot_iterator().get_next()

        # labels= tf.reshape(labels,shape=(batch_size,1))
        return features, labels
        # return ds.make_one_shot_iterator()

        
        
    def get_training_batch(self,batch_size,num_epochs,features=None):

        training_batch = self.pipeline_feeder(self.training_features,
                                           self.training_targets,
                                                   batch_size=batch_size,
                                                   num_epochs=num_epochs,
                                                   shuffle=True )
        return training_batch

    def get_testing_batch(self,num_epochs,features=None):
        testing_batch =  self.pipeline_feeder(self.testing_features,
                                                   self.testing_targets,
                                                           num_epochs=num_epochs,
                                                           batch_size=self.testing_targets.shape[0], # All the testing set
                                                           shuffle=False)
        return testing_batch

    
    def get_validation_batch(self,num_epochs,features=None):
        validation_batch =  self.pipeline_feeder(self.validation_features,
                                                   self.validation_targets,
                                                           num_epochs=num_epochs,
                                                           batch_size=self.validation_targets.shape[0], # All the validation set
                                                           shuffle=False)
        return validation_batch
    
    def get_all_dataset(self,features=None):
        dataset_batch = self.pipeline_feeder(self.allDataset_features,
                                                             self.allDataset_targets,
                                                             num_epochs=1,
                                                             batch_size=self.allDataset_targets.shape[0],
                                                             shuffle=False)

        return dataset_batch




C:\Users\Maturrin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
features= [ 'time', 'pos/0', 'pos/1','spd/0', 'spd/1']
features_aux= [ 'type', 'sender', 'messageID', 'pos/2','spd/0', 'spd/1', 'pos/2']
# Most of the features are discrete and do not represent continous values
# There is one exception with pos/2 that is continous, however it does not change, it can be eliminated 

targets= ['classs']


In [10]:
dataset = dataset_setup(features,targets)

In [11]:
dataset.input_data("data1.csv")

Training samples  (3050, 10) 
Validation samples (323, 10) 
Testing samples (656, 10)


In [12]:
features,target = dataset.get_training_batch(1,10)
import tensorflow as tf

sess =  tf.InteractiveSession()
for i in range(10 ) : print(sess.run(features))
sess.close()

(3050, 5) (3050, 1)
[[ 0.18187908 -0.07851001 -0.28588466 -0.03332444  0.0124303 ]]
[[ 0.39126259 -0.5319003  -0.06187343  0.26559908 -0.02362654]]
[[ 0.47190416 -0.06703684 -0.30836746  0.1818891  -0.05342056]]
[[-0.50006901  0.10855415 -0.25187078 -0.03332444  0.0124303 ]]
[[-0.24528347 -0.28335894  0.28942064 -0.03386514  0.3649975 ]]
[[ 0.23449413  0.11048    -0.26164314 -0.03332444  0.0124303 ]]
[[-0.01052427 -0.31280695 -0.43148568 -0.01688538  0.10195898]]
[[ 0.19071824  0.05520614 -0.1545439   0.28631738  0.02339907]]
[[-0.15662801  0.34518099  0.19399679 -0.03332444  0.0124303 ]]
[[-0.48776965 -0.27834741  0.51235021 -0.1123976  -0.42218458]]


In [13]:
features,target=dataset.get_validation_batch(10)
features,target=dataset.get_testing_batch(10)
features,target=dataset.get_all_dataset(10)

(323, 5) (323, 1)
(656, 5) (656, 1)
(4029, 5) (4029, 1)
